<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
## Open Machine Learning Course
<center>Auteur: [Yury Kashnitsky](https://www.linkedin.com/in/festline/), Data Scientist chez Mail.ru Group <br>
    <center> Edité et traduit par [Cissé Ousmane](https://github.com/oussou-dev) <br>
    Tout le contenu est distribué sous la licence [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/).
Vous pouvez utiliser ce matériel à n'importe quelle fin (vous pouvez éditer, corriger et utiliser comme exemple) sauf un usage commercial avec mention obligatoire de l'auteur.</center></center></center>

# <center> Misson n ° 6 (démo).
## <center> Exploration de OLS (Moindres carrés ordinaires), Lasso et forêt aléatoire <br> dans une problème de régression
    
<img src="https://habrastorage.org/webt/-h/ns/aa/-hnsaaifymavmmudwip9imcmk58.jpeg" width="30%">

**Remplissez le code manquant et choisissez les réponses dans ce [formulaire](https://docs.google.com/forms/d/1aHyK58W6oQmNaqEfvpLTpo6Cb0-ntnvJ18rZcvclkvw/edit).**</center></center>

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.metrics.regression import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, Lasso
from sklearn.ensemble import RandomForestRegressor

**Nous travaillons avec le jeu de données de qualité UCI Wine (inutile de le télécharger: il est déjà disponible, dans le référentiel de cours et dans le jeu de données Kaggle).**

In [4]:
data = pd.read_csv('../input/winequality-white.csv')

In [ ]:
data.head()

In [ ]:
data.info()

**Séparez la caractéristique cible, divisez les données en proportion de 7:3 (30% forment le holdout set (jeu de données de validation), utilisez random_state = 17), et prétraitez les données avec `StandardScaler`.**

In [ ]:
y = data['quality']
X = data.drop('quality', axis=1)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, 
                                                          random_state=17)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Régression linéaire

**Entraîner un modèle de régression linéaire simple (Moindres carrés ordinaires).**

In [ ]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train);

**<font>Question 1: </font> Quelles sont les prédictions des erreurs quadratiques moyennes sur les données d'entraînement et les données de validation?**

In [ ]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

**Triez les caractéristiques en fonction de leur influence sur la caractéristique cible : qualité du vin (wine quality). Attention, les grands coefficients positifs et les grands coefficients négatifs signifient une influence importante sur la cible. C'est pratique d'utiliser `pandas.DataFrame` ici.**

**<font>Question 2: </font> Quelle caractéristique ce modèle de régression linéaire considère-t-il comme la plus influente sur la qualité du vin (wine quality)?**

In [ ]:
linreg_coef = pd.DataFrame({'coef': linreg.coef_, 'coef_abs': np.abs(linreg.coef_)},
                          index=data.columns.drop('quality'))
linreg_coef.sort_values(by='coef_abs', ascending=False)

## Régression Lasso

**Entraîner un modèle LASSO avec $\alpha = 0.01$ (régularisation faible) et des données mises à l’échelle. De nouveau, définissez random_state = 17.**

In [ ]:
lasso1 = Lasso(alpha=0.01, random_state=17)
lasso1.fit(X_train_scaled, y_train)

**Quelle caractéristique est la moins informative pour prédire la qualité du vin, selon ce modèle LASSO?**

In [ ]:
lasso1_coef = pd.DataFrame({'coef': lasso1.coef_, 'coef_abs': np.abs(lasso1.coef_)},
                          index=data.columns.drop('quality'))
lasso1_coef.sort_values(by='coef_abs', ascending=False)

**Entraîner LassoCV avec random_state = 17 pour choisir la meilleure valeur de $\alpha$ dans une validation croisée sur 5-fold.**

In [ ]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(random_state=17, cv=5, alphas=alphas)
lasso_cv.fit(X_train_scaled, y_train)

In [ ]:
lasso_cv.alpha_

**<font>Question 3: </font> Quelle caractéristique est la moins informative pour prévoir la qualité du vin, selon le modèle LASSO ajusté (avec réglages)?**

In [ ]:
lasso_cv_coef = pd.DataFrame({'coef': lasso_cv.coef_, 'coef_abs': np.abs(lasso_cv.coef_)},
                          index=data.columns.drop('quality'))
lasso_cv_coef.sort_values(by='coef_abs', ascending=False)

**<font>Question 4: </font> Quelles sont les erreurs quadratiques moyennes des prédictions LASSO ajustées sur les données d'entraînement et de validation?**

In [ ]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

## Forêt aléatoire

**Entraînez une forêt aléatoire avec des paramètres prêts à l'emploi, en définissant uniquement random_state sur 17.**

In [ ]:
forest = RandomForestRegressor(random_state=17)
forest.fit(X_train_scaled, y_train)

**<font>Question 5: </font> Quelles sont les erreurs quadratiques moyennes du modèle RF sur le jeu d'entraînement, en validation croisée (cross_val_score with scoring = 'neg_mean_squared_error' et d'autres arguments laissés avec des valeurs par défaut) et sur les données de validation?**

In [ ]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, forest.predict(X_train_scaled)))
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(forest, X_train_scaled, y_train, 
                                                                       scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, forest.predict(X_holdout_scaled)))

**Réglez les hyperparamètres `max_features` et` max_depth` avec GridSearchCV et vérifiez à nouveau la moyenne des validations croisées entre MSE et MSE sur le jeu de données de validation.**

In [ ]:
forest_params = {'max_depth': list(range(10, 25)), 
                  'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=17), 
                                 forest_params, 
                                 scoring='neg_mean_squared_error',  
                                 n_jobs=-1, cv=5,
                                  verbose=True)
locally_best_forest.fit(X_train_scaled, y_train)

In [ ]:
locally_best_forest.best_params_, locally_best_forest.best_score_

**<font>Question 6: </font> Quelles sont les erreurs quadratiques moyennes du modèle RF ajusté en validation croisée (cross_val_score with scoring = 'neg_mean_squared_error' et autres arguments laissés avec des valeurs par défaut) et sur les données de validation?**

In [ ]:
print("Mean squared error (cv): %.3f" % np.mean(np.abs(cross_val_score(locally_best_forest.best_estimator_,
                                                        X_train_scaled, y_train, 
                                                        scoring='neg_mean_squared_error'))))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, 
                                                             locally_best_forest.predict(X_holdout_scaled)))

**Montrer l’importance des caractéristiques de RF. Encore une fois, il est préférable de l'afficher comme un DataFrame.** <br>
**<font>Question 7: </font> Quelle est la caractéristique la plus importante selon le modèle Random Forest?**

In [ ]:
rf_importance = pd.DataFrame(locally_best_forest.best_estimator_.feature_importances_, 
                             columns=['coef'], index=data.columns[:-1]) 
rf_importance.sort_values(by='coef', ascending=False)

**Tirer des conclusions sur la performance des 3 modèles explorés dans cette misson de prédiction particulière.**

La dépendance de la qualité du vin par rapport à d'autres caractéristiques est, on peut le supposer, non linéaire. Random Forest fonctionne donc mieux dans cette mission.